In [2]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

# Loading the formatted train datset

In [4]:
train=pd.read_csv('formattedtrain.csv')
train.head()

,Unnamed: 0,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,60,3,65.0,8450,1,3,3,0,4,...,0,0,0,0,0,2,2008,8,4,208500
1,1,20,3,80.0,9600,1,3,3,0,2,...,0,0,0,0,0,5,2007,8,4,181500
2,2,60,3,68.0,11250,1,0,3,0,4,...,0,0,0,0,0,9,2008,8,4,223500
3,3,70,3,60.0,9550,1,0,3,0,0,...,272,0,0,0,0,2,2006,8,0,140000
4,4,60,3,84.0,14260,1,0,3,0,2,...,0,0,0,0,0,12,2008,8,4,250000


Now iam separating/splitting train data into 2parts, train and test 70:30 %nd from this dataset_train dataset.
This is necessary for validation of our model, because in actual test.csv file we don't have target column and we are predicting using this model.
The prediction is done after validation of our model through train dataset.



In [5]:
train_data, test_data = np.split(train.sample(frac=1, random_state=700),
                                 [int(0.7 * len(train))])


# train_data.to_csv("train1.csv", header=False, index=False)
# validation_data.to_csv("validation.csv", header=False, index=False)

In [6]:
train_data.shape, test_data.shape 

((995, 76), (427, 76))

Now, In general, the model built in AWS sagemaker,it is always necessary to put target column "SalePrice" as the first column.
So we are making the same format in case of splitted train and test data.

In [9]:
# setting the model path or output path in our s3 bucket where we will store our formatted files
bucket_name = 'modeltestbucket'
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://modeltestbucket/xgboost-as-a-built-in-algo/output


In [ ]:
# for train datset
pd.concat([train_data['SalePrice'], train_data.drop(['SalePrice'], axis=1)],
          axis=1).to_csv('train.csv', index=False,header=True)



boto3.Session().resource('s3').Bucket('modeltestbucket').Object(os.path.join(prefix,'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format('modeltestbucket',prefix), content_type='csv')

In [ ]:
# for test/validation dataset
pd.concat([test_data['SalePrice'], test_data.drop(['SalePrice'], axis=1)],
          axis=1).to_csv('test.csv', index=False,header=True)



boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation'.format(bucket_name, prefix), content_type='csv')

# Model building by xgboost algorithm present in sagemaker

In sagemaker, the inbuilt algrithms like xgboost is present in the container/images, and needs to be pulled that container to 
sagemaker instance

In [94]:
container = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [125]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"reg:Linear",
        "num_round":50
        }

In [126]:
estimator = sagemaker.estimator.Estimator(image_uri=container,
                                          hyperparameters=hyperparameters,
                                          role='AmazonSageMaker-ExecutionPolicy-20220616T102885',
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [127]:
# estimator.fit({'train': s3_input_train,
#                'validation': s3_input_test})



estimator.fit( {'train':'s3://modeltestbucket/xgboost-as-a-built-in-algo/train/train.csv',
              'validation': 's3://modeltestbucket/xgboost-as-a-built-in-algo/test/test.csv'})

ResponseParserError: Unable to parse response (no element found: line 1, column 0), invalid XML received. Further retries may succeed:
b''

In [ ]:
# splitting the features and labels

In [26]:
y=train['SalePrice']
x=train.drop('SalePrice', axis=1)

In [27]:
x.shape, y.shape

((1422, 74), (1422,))

# Take Test data

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.2,random_state=40)

In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [30]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Building the Model and checking the best accuracy

In [33]:
def do_prediction(classifier):
    
    # training the classifier on the dataset
    classifier.fit(X_train, y_train)
    
    #Do prediction and evaluting the prediction
    prediction = classifier.predict(X_test)
    cross_validation_score = cross_val(X_train, y_train, classifier)
    error = mean_absolute_error(y_test, prediction)
    return error, cross_validation_score

def cross_val(xtrain, ytrain, classifier):
    
    # Applying k-Fold Cross Validation
    accuracies = cross_val_score(estimator = classifier, X = xtrain, y = ytrain, cv = 5)
    return accuracies.mean()

# DecisionTreeRegressor

In [34]:
model_2 = DecisionTreeRegressor()
error, score = do_prediction(model_2)

print('Decision Tree Regressor MAE: {}'.format(round(error,2)))
print('Cross validation score: {}'.format(round(score,2)))

Decision Tree Regressor MAE: 24959.48
Cross validation score: 0.68


# RandomForestRegressor

In [35]:
model_3 = RandomForestRegressor()
error, score = do_prediction(model_3)

print('Random Forest Regressor MAE: {}'.format(round(error,2)))
print('Cross validation score: {}'.format(round(score,2)))

Random Forest Regressor MAE: 15706.5
Cross validation score: 0.83


# Best score is from RandomForestRegressormodel

In [36]:
best_model = RandomForestRegressor(n_estimators=300,
                                  min_samples_split=10,
                                  min_samples_leaf=1,
                                  max_features='sqrt',
                                  max_depth=60,
                                  bootstrap=False)

error, score = do_prediction(best_model)
print('Random Forest with hyperparameter tuning MAE: {}'.format(round(error,2)))
print('Cross validation score: {}'.format(round(score,2)))

Random Forest with hyperparameter tuning MAE: 16213.33
Cross validation score: 0.85


In [37]:
y_pred=best_model.predict(X_test)

# Actual and Predicted comparision

In [38]:
price = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
price.reset_index(inplace=True,drop=True)
price

,Actual,Predicted
0,120500,123232.516069
1,221000,204795.729124
2,181000,177928.561690
3,165000,177722.904081
4,167500,150286.871316
...,...,...
280,217000,205155.964624
281,172785,171979.680241
282,180000,239359.470737
283,171000,167332.315616


# Saving the model

In [41]:
import pickle
# open a file, where you ant to store the data
file = open('best_model.pkl', 'wb')

# dump information to that file
pickle.dump(best_model, file)

# close the file
file.close()

# Now Testing the model with our test.csv file and predict the outputs.



Removing the null values of test.csv and converting categoriacl to numerical columns

In [12]:
path2="C:\\Users\\santh\\S3 Bucket to Sage Maker-AWS\\Data\\test-2.csv"

test=pd.read_csv(path2)

test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [42]:
test['LotFrontage']=test['LotFrontage'].fillna(test['LotFrontage'].mean())
test.drop(['Alley'],axis=1,inplace=True)
test['BsmtCond']=test['BsmtCond'].fillna(test['BsmtCond'].mode()[0])
test['BsmtQual']=test['BsmtQual'].fillna(test['BsmtQual'].mode()[0])
test['FireplaceQu']=test['FireplaceQu'].fillna(test['FireplaceQu'].mode()[0])
test['GarageType']=test['GarageType'].fillna(test['GarageType'].mode()[0])
test.drop(['GarageYrBlt'],axis=1,inplace=True)
test['GarageFinish']=test['GarageFinish'].fillna(test['GarageFinish'].mode()[0])
test['GarageQual']=test['GarageQual'].fillna(test['GarageQual'].mode()[0])
test['GarageCond']=test['GarageCond'].fillna(test['GarageCond'].mode()[0])
test.drop(['PoolQC','Fence','MiscFeature'],axis=1,inplace=True)
test.drop(['Id'],axis=1,inplace=True)
test['MasVnrType']=test['MasVnrType'].fillna(test['MasVnrType'].mode()[0])
test['MasVnrArea']=test['MasVnrArea'].fillna(test['MasVnrArea'].mode()[0])
test['BsmtExposure']=test['BsmtExposure'].fillna(test['BsmtExposure'].mode()[0])
test['BsmtFinType2']=test['BsmtFinType2'].fillna(test['BsmtFinType2'].mode()[0])
test.dropna(inplace=True)

In [44]:
cat_bf=test.select_dtypes(exclude=np.number)
cat_bf.columns

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition'],
      dtype='object')

In [45]:
lb=LabelEncoder()
for i in cat_bf.columns:
    test[i]=lb.fit_transform(np.array(test[i]).reshape(-1,1))

D:\Anaconda\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [48]:
test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,2,80.0,11622,1,3,3,0,4,0,...,0,0,0,120,0,0,6,2010,8,4
1,20,3,81.0,14267,1,0,3,0,0,0,...,36,0,0,0,0,12500,6,2010,8,4
2,60,3,74.0,13830,1,0,3,0,4,0,...,34,0,0,0,0,0,3,2010,8,4
3,60,3,78.0,9978,1,0,3,0,4,0,...,36,0,0,0,0,0,6,2010,8,4
4,120,3,43.0,5005,1,0,1,0,4,0,...,82,0,0,144,0,0,1,2010,8,4


# loading the saved model

In [46]:

# open a file, where you stored the pickled data
file = open('best_model.pkl', 'rb')

# dump information to that file
modelfile= pickle.load(file)

# close the file
file.close()


In [47]:
modelfile

RandomForestRegressor(bootstrap=False, max_depth=60, max_features='sqrt',
                      min_samples_split=10, n_estimators=300)

In [49]:
testarray=np.array(test)

In [50]:
testarray

array([[2.000e+01, 2.000e+00, 8.000e+01, ..., 2.010e+03, 8.000e+00,
        4.000e+00],
       [2.000e+01, 3.000e+00, 8.100e+01, ..., 2.010e+03, 8.000e+00,
        4.000e+00],
       [6.000e+01, 3.000e+00, 7.400e+01, ..., 2.010e+03, 8.000e+00,
        4.000e+00],
       ...,
       [2.000e+01, 3.000e+00, 1.600e+02, ..., 2.006e+03, 8.000e+00,
        0.000e+00],
       [8.500e+01, 3.000e+00, 6.200e+01, ..., 2.006e+03, 8.000e+00,
        4.000e+00],
       [6.000e+01, 3.000e+00, 7.400e+01, ..., 2.006e+03, 8.000e+00,
        4.000e+00]])

In [51]:
modelpredictions=modelfile.predict(testarray)

In [52]:
modelpredictions

array([375393.21543254, 379499.22865873, 435513.02324206, ...,
       366774.89196693, 330308.40612698, 451779.55941667])